# Étude du cas de Sod (tube à choc)

`Python` sera utilisé ici comme `matlab`. Des fonctionnalités supplémentaires peuvent être ajoutées par l'import de modules, standards à une distribution (comme `math`, `numpy`) ou personnalisés comme ci-dessous. Des fonctionnalités d'édition sont propres à [`Ipython/Notebook`](#ipython).

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
plt.rcParams['font.size'] = 14 ; plt.rcParams['axes.grid'] = True ; plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['lines.linewidth'] = 1.5 ; plt.rcParams['legend.fontsize'] = 12
#
import pyfvm.mesh  as mesh
import pyfvm.model as model
import pyfvm.field as data
from pyfvm.xnum        import *
from pyfvm.integration import *
import sod
#print plt.rcParams

On cherche à résoudre l'évolution instationnaire du problème linéaire de convection suivant

$$ \frac{\partial q}{\partial t} + a \frac{\partial q}{\partial x} = 0 $$

pour la quantité transportée $q(x,t)$ et la condition intiale $q_0(x)$ sur le domaine $[0;\ell]$ avec des conditions périodiques. On choisit $\ell=1\rm~m$ et $a=1\rm~m/s$. 


## Définition des maillages, du modèle physique et solution initiales


In [ ]:
meshref = mesh.unimesh(ncell=1000, length=1.)
mesh100 = mesh.unimesh(ncell=100, length=1.)
mesh50  = mesh.unimesh(ncell=50,  length=1.)

mymodel = model.eulermodel()

#boundary condition bc : type of boundary condition - "p"=periodic / "d"=Dirichlet
bc       = 'd'
bcvalues = []
neq      = 3

for i in range(neq+1):
    bcvalues.append(np.zeros(2))

# Left Boundary

bcvalues[0][0] = 1.0      # density  rho
bcvalues[1][0] = 0.0      # velocity u       
bcvalues[2][0] = 2.5      # int. nrg e            
bcvalues[3][0] = 1.0      # pressure p            

# Right Boundary

bcvalues[0][1] = 0.125    # density  rho            
bcvalues[1][1] = 0.0      # velocity u            
bcvalues[2][1] = 2.0      # int. nrg e             
bcvalues[3][1] = 0.1      # pressure p            

gamma      = 1.4


In [ ]:
# Set of computations
endtime = 0.25
cfls    = [ 0.8 ]
# extrapol1(), extrapol2()=extrapolk(1), centered=extrapolk(-1), extrapol3=extrapolk(1./3.)
#xmeths  = [ extrapol1(), extrapol2(), centered(), extrapol3() ]
xmeths  = [ extrapol1(), muscl() ]
# explicit, rk2, rk3ssp, rk4, implicit, trapezoidal=cranknicolson
tmeths  = [ rk4 ]
#legends = [ 'O1 upwind', 'O2 upwind', 'O2 centered', 'O3 extrapol' ]
legends = [ 'O1','O2 muscl' ]

meshs      = [ mesh100 ]
initm      = sod.initSod
exactPdata = sod.exactSod(meshref,endtime)

solvers = []
results = []

nbcalc  = max(len(cfls), len(tmeths), len(xmeths), len(meshs))

for i in range(nbcalc):
    field0 = scafield(mymodel, bc, (meshs*nbcalc)[i].ncell, bcvalues)
    field0.qdata = initm((meshs*nbcalc)[i])
    solvers.append((tmeths*nbcalc)[i]((meshs*nbcalc)[i], (xmeths*nbcalc)[i]))
    start = time.clock()
    results.append(solvers[-1].solve(field0, (cfls*nbcalc)[i], np.array([endtime])))
    print "cpu time of "+"%-11s"%(legends[i])+" computation (",solvers[-1].nit,"it) :",time.clock()-start,"s"

In [ ]:
fig, ax = plt.subplots(2,2, figsize=[18,12])

# Figure
refstyle = 'k-'
markers = ['o', 'x', 'D', '*', '+', '>', '<', 'd']
markerstyle = {'color': 'black', 'markerfacecolor': 'none', 'markersize':5, 'markeredgewidth': 1.2, 'alpha': 1.0, 'markeredgecolor': 'blue'}
#
labels = ["exact solution"]
for i in range(nbcalc):
    labels.append(legends[i]+", t=%.2f"%results[i][0].time)
# Density
#
#fig.suptitle('Density profile along the Sod shock-tube, CFL %.3f'%cfls[0], fontsize=12, y=0.93)
# Exact solution
ax[0][0].plot(meshref.centers(), exactPdata[0], refstyle)
# Numerical solution
for i in range(nbcalc):
    rho=results[i][0].qdata[0]
    ax[0][0].plot((meshs*nbcalc)[i].centers(), rho, markers[i], **markerstyle)
ax[0][0].legend(labels, loc='best',prop={'size':10})
# 
# VELOCITY
#
#fig.suptitle('Velocity profile along the Sod shock-tube, CFL %.3f'%cfls[0], fontsize=12, y=0.93)
# Exact solution
ax[1][0].plot(meshref.centers(), exactPdata[1], refstyle)
# Numerical solution
for i in range(nbcalc):
    u = results[i][0].qdata[1]/results[i][0].qdata[0] 
    ax[1][0].plot((meshs*nbcalc)[i].centers(), u, markers[i], **markerstyle)
ax[1][0].legend(labels, loc='best',prop={'size':10})
#
# INTERNAL ENERGY
#
#fig.suptitle('Internal Energy profile along the Sod shock-tube, CFL %.3f'%cfls[0], fontsize=12, y=0.93)
# Exact solution
ax[0][1].plot(meshref.centers(), exactPdata[2], refstyle)
# Numerical solution
for i in range(nbcalc):
    e = (results[i][0].qdata[2]-0.5*results[i][0].qdata[1]**2/results[i][0].qdata[0])/results[i][0].qdata[0]
    ax[0][1].plot((meshs*nbcalc)[i].centers(), e, markers[i], **markerstyle)
ax[0][1].legend(labels, loc='best',prop={'size':10})
#
# PRESSURE
#
#fig.suptitle('Pressure profile along the Sod shock-tube, CFL %.3f'%cfls[0], fontsize=12, y=0.93)
# Exact solution
ax[1][1].plot(meshref.centers(), exactPdata[3], refstyle) 
# Numerical solution
for i in range(nbcalc):
    p = (gamma-1.0)*(results[i][0].qdata[2]-0.5*results[i][0].qdata[1]**2/results[i][0].qdata[0])
    ax[1][1].plot((meshs*nbcalc)[i].centers(), p, markers[i], **markerstyle)
ax[1][1].legend(labels, loc='best',prop={'size':10})
                           

## Calcul et comparaison

---

<a id="ipython"></a>
## Ipython et notebook : usage

* le notebook utilise la langage de base python en version améliorée, Ipython, qui permet la complétion des noms (variables, fonctions, modules) avec la touche tabulation
* toutes les cellules peuvent être modifiées par un double-clic et sont réinterprêtées avec `shift-entrée`
* l'ensemble de la feuille peut être exécutée avec le menu `Cell/run all cells`
* **n'oubliez pas de sauvegarder régulièrement votre feuille** (bouton _enregistrer_)


In [ ]:
from IPython.core.display import HTML ; HTML(open("../custom.css", "r").read()) # notebook style